In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn, time
import bnlearn as bn # used for creating the Baysian Network structure
import time # for timing how long it takes to generate the synthetic data
import random # for sampling
import itertools # for generating combinations
import json # for saving and loading data

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://github.com/jnear/cs3110-data-privacy/raw/main/homework/adult_with_pii.csv')

In [2]:
# Dropping some of the columns
adult = adult.drop(['Name', 'DOB', 'SSN', 'Zip', 'fnlwgt'], axis=1)

In [3]:
# inputs: a dataset in a pandas dataframe df
#         n, an integer for number of synthetic rows
#         epsilon, the amount of your privacy budget you want to use

def create_synthetic_data(df, n, epsilon):
    
    # Functions used in this implementation:
    
    def prune_edges(dag):
        # find a topological ordering of the dag nodes
        topo_nodes = bn.topological_sort(dag)

        # get the edges of the dag for altering
        all_edges = dag['model_edges']

        to_nodes = []
        min_edges = []

        for node in topo_nodes:
            for edge in all_edges:
                if edge[0] == node and (edge[1] not in to_nodes):
                    # add edge[1] to to_nodes
                    to_nodes.append(edge[1])
                    # add the edge to min_edges
                    min_edges.append(edge)

        return min_edges
    
    
    def dp_two_marginal(df, col1, col2, epsilon):
    
        # create noisy contingency table
        ct = df[[col1, col2]].value_counts()
        noisy_ct = ct.apply(lambda x: laplace_mech(x, 1, epsilon))

        non_negative_noisy_ct = np.clip(noisy_ct, 0, None)
        total = np.sum(np.sum(non_negative_noisy_ct))

        marginals = non_negative_noisy_ct / total

        return marginals.to_frame(name='probability').reset_index()

    # From a list of edges, create a dictionary of marginals
    def create_marginals(df, edges, epsilon):
        marginal_dict = {}

        epsilon_frac = len(edges)

        for edge in edges:
            marginal_dict[edge] = dp_two_marginal(df, edge[0], edge[1], epsilon_frac)

        return marginal_dict

    
    def gen_samples(n, marginal):
        samples = marginal.sample(n=n,
                                  replace=True,
                                  weights='probability')
        return samples
    
    
    def gen_col_val(causal_col, causal_val, target_col, marginal):

            # create a new marginal based on the given value for the causal_column
            conditional_marginal = marginal[marginal[causal_col] == causal_val]
            total = conditional_marginal['probability'].sum()
            conditional_marginal['probability'] / total

            target_val = conditional_marginal.sample(1, replace=True, weights='probability')[target_col].iloc[0]

            return target_val
    
    # first we create a directed acyclical graph that has
    # estimated causal relations between the columns
    # this uses the bnlearn library's structure_learning class
    df_dag = bn.structure_learning.fit(df, methodtype='hc', scoretype='bic')
    
    # this graph might have too many edges for out needs so we will prune
    # some of the edges to do this it helpful to sort the nodes in the DAG
    # into a topological ordering (this is done within the function called)
    edges = prune_edges(df_dag)
    
    # create the marginals
    marginals = create_marginals(df, edges, epsilon)
    
    # create a new dataframe for the samples
    synthetic_data = pd.DataFrame()
    
    # start at the top of the order
    for edge in edges:
        
        # 1. Our dataset is currently empty
        # Create the first 2 columns of data
        if synthetic_data.empty:
            synthetic_data = gen_samples(n, marginals[edge]).drop(['probability'], axis=1)
        
        # 2. We are creating a column that branches off an existing column
        # create a conditional marginal using the value in edge[0]
        # generate values for the new column using this data
        elif edge[0] in synthetic_data.columns:
            synthetic_data[edge[1]] = synthetic_data.apply(lambda row: \
                                                                     gen_col_val(edge[0], row[edge[0]], edge[1], marginals[edge]), \
                                                                     axis = 1)
        
        # 3. We are creating 2 new columns, one of which is a root, but we have already generated some columns
        # use the two-way marginal to create 2 new columns and merge to the data
        else:
            # we can create the data using gen_samples and just merge it with the rest of the data
            new_columns = gen_samples(n, marginals[edge]).drop(['probability'], axis=1)
            # merge new columns with synthetic-data
            synthetic_data = pd.concat([synthetic_data.reset_index(drop=True), new_columns.reset_index(drop=True)], axis=1)
    
    return synthetic_data, edges, df_dag

In [84]:
### This takes a few minutes to run, instead run the cell that loads the data

time_start = time.time()
adult_synth, edges, adult_dag = create_synthetic_data(adult, len(adult), 1.0)
time_end = time.time()
time_taken = time_end - time_start
print(time_taken, ' seconds to create ', len(adult), ' synthetic rows.')

[bnlearn] >Warning: Computing DAG with 14 nodes can take a very long time!
[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bic]
[bnlearn] >Compute structure scores for model comparison (higher is better).
210.1876049041748  seconds to create  32561  synthetic rows.


In [89]:
### Save the synthetic data dataframe, edge list and DAG to file

adult_synth.to_csv('adult_synth', sep=',', index=False, encoding='utf-8')
with open("edges.json", 'w') as f:
    json.dump(edges, f, indent=2)

In [4]:
### Load the data here
adult_synth = pd.read_csv('adult_synth')

with open("edges.json", 'r') as f:
    edges = json.load(f)

edge_ = []
for edge in edges:
    edge_.append((edge[0], edge[1]))
edges = edge_

## Accuracy
To test the accuracy of the synthetic data I will employ several metrics:
  - compare the counts of the synthetic data with the original data and 
  - compare 10 random set of 2-way contingencies
  - compare 5 random set of 3-way contingencies

In [5]:
# This measures the percent error that occurs in the synthetic data as compared to the
# original data as far as the counts in each column

def compare_counts(synth_df, df):
    avg_error_df = pd.Series()
    for column in df.columns:
        df_counts = df[column].value_counts() 
        synth_df_counts = synth_df[column].value_counts()
        combined_df = df_counts.to_frame().join(other=synth_df_counts, how = 'left', sort = True, lsuffix=' original', rsuffix=' synthetic').fillna(0)
        combined_df['Percent Error'] = pct_error(combined_df[column + ' original'], combined_df[column + ' synthetic'])
        avg_error_df[column] = combined_df['Percent Error'].sum()/len(combined_df['Percent Error'])
    return avg_error_df

In [6]:
compare_counts(adult_synth, adult)

Age               11.358092
Workclass         20.851286
Education          3.294521
Education-Num      3.294521
Marital Status     4.335595
Occupation         5.932392
Relationship       2.814341
Race               3.977370
Sex                1.824359
Capital Gain      31.318485
Capital Loss      35.349208
Hours per week    19.637780
Country           12.338605
Target             0.260381
dtype: float64

In [7]:
# synth_df is a dataframe of synthetic data and df the dataframe it was created from
# columns is a list of column names
def compare_contingency_counts(synth_df: pd.DataFrame, df: pd.DataFrame, columns: list[str]):
    
    df_counts = adult[columns].value_counts().rename('actual count')
    synth_df_counts = adult_synth[columns].value_counts().rename('synthetic count')
    combined_df = df_counts.to_frame().join(other=synth_df_counts, how = 'left', sort = True).fillna(0)
    combined_df['Percent Error'] = pct_error(combined_df['actual count'], combined_df['synthetic count'])
    avg_error = combined_df['Percent Error'].sum()/len(combined_df['Percent Error'])
    
    return avg_error

In [9]:
# Now let's check how accurate we are across all possible corellations
# As there are 14 columnns this gives us 91 possible combinations (14 choose 2)

all_combinations = list(itertools.combinations(adult.columns, 2))

error_two_columns = [compare_contingency_counts(adult_synth, adult, list(column_pair)) for column_pair in all_combinations]

all_combinations_indices = ["_".join(columns) for columns in all_combinations]

error_series_two = pd.Series(error_two_columns, all_combinations_indices)
error_series_two.describe()

count       91.000000
mean       411.114731
std       3255.113264
min          3.294521
25%         35.860593
50%         61.666821
75%         75.978452
max      31107.820575
dtype: float64

In [11]:
error_series_two[error_series_two > 100]

Age_Relationship                 214.263463
Age_Target                       162.247019
Marital Status_Relationship      921.840436
Marital Status_Target            101.910705
Occupation_Target                140.357746
Relationship_Sex               31107.820575
Relationship_Capital Gain        118.394799
Relationship_Target              217.407822
dtype: float64

In [12]:
error_series_two[error_series_two < 100][error_series_two > 50]

Age_Workclass                    80.860934
Age_Education                    97.584902
Age_Education-Num                97.584902
Age_Occupation                   75.074769
Age_Capital Gain                 75.473501
Age_Capital Loss                 75.244264
Age_Hours per week               87.406668
Age_Country                      72.398276
Workclass_Race                   63.766136
Workclass_Capital Gain           74.862238
Workclass_Capital Loss           70.042995
Workclass_Hours per week         90.530002
Workclass_Country                84.680381
Education_Capital Gain           75.978452
Education_Capital Loss           67.535367
Education_Hours per week         62.604964
Education_Country                83.930000
Education-Num_Capital Gain       75.978452
Education-Num_Capital Loss       67.535367
Education-Num_Hours per week     62.604964
Education-Num_Country            83.930000
Marital Status_Capital Gain      74.359167
Marital Status_Capital Loss      68.104253
Marital Sta

In [13]:
error_series_two[error_series_two < 50][error_series_two > 10]

Age_Marital Status              25.074081
Age_Race                        43.973056
Age_Sex                         15.151349
Workclass_Education             45.284310
Workclass_Education-Num         45.284310
Workclass_Marital Status        42.330613
Workclass_Occupation            18.486470
Workclass_Relationship          41.399406
Workclass_Sex                   36.472935
Workclass_Target                24.697882
Education_Marital Status        44.859407
Education_Occupation            22.288185
Education_Relationship          47.275552
Education_Race                  49.406675
Education_Sex                   17.663802
Education-Num_Marital Status    44.859407
Education-Num_Occupation        22.288185
Education-Num_Relationship      47.275552
Education-Num_Race              49.406675
Education-Num_Sex               17.663802
Marital Status_Occupation       30.779868
Marital Status_Race             34.248054
Occupation_Relationship         11.485707
Occupation_Race                 38

In [14]:
error_series_two[error_series_two <= 10]

Education_Education-Num    3.294521
Education_Target           7.191246
Education-Num_Target       7.191246
Marital Status_Sex         4.169578
Occupation_Sex             7.625550
dtype: float64

In [15]:
edges

[('Education-Num', 'Education'),
 ('Education', 'Occupation'),
 ('Education', 'Target'),
 ('Occupation', 'Sex'),
 ('Occupation', 'Workclass'),
 ('Occupation', 'Relationship'),
 ('Sex', 'Marital Status'),
 ('Marital Status', 'Age'),
 ('Country', 'Race'),
 ('Target', 'Capital Gain'),
 ('Target', 'Hours per week'),
 ('Target', 'Capital Loss')]

In [16]:
error_dag = [compare_contingency_counts(adult_synth, adult, list(column_pair)) for column_pair in edges]

error_dag_indices = ["_".join(columns) for columns in edges]

error_series_dag = pd.Series(error_dag, error_dag_indices)
error_series_dag.describe()

count    12.000000
mean     19.471737
std      12.791880
min       3.294521
25%       7.516974
50%      20.387328
75%      27.583899
max      41.334069
dtype: float64

In [17]:
error_series_dag

Education-Num_Education     3.294521
Education_Occupation       22.288185
Education_Target            7.191246
Occupation_Sex              7.625550
Occupation_Workclass       18.486470
Occupation_Relationship    11.485707
Sex_Marital Status          4.169578
Marital Status_Age         25.074081
Country_Race               41.334069
Target_Capital Gain        31.026969
Target_Hours per week      26.436209
Target_Capital Loss        35.248252
dtype: float64

In [18]:
# For the combinations of three columns:

all_combinations = list(itertools.combinations(adult.columns, 3))

error_three_columns = [compare_contingency_counts(adult_synth, adult, list(column_pair)) for column_pair in all_combinations]

all_combinations_indices = ["_".join(columns) for columns in all_combinations]

error_series_three = pd.Series(error_three_columns, all_combinations_indices)
error_series_three.describe()

count      364.000000
mean       182.549827
std        857.791798
min          7.191246
25%         72.820799
50%         80.773372
75%         90.403688
max      14437.413050
dtype: float64

In [19]:
error_series_three[error_series_three > 100]

Age_Workclass_Relationship                      127.412900
Age_Workclass_Target                            121.781622
Age_Marital Status_Relationship                 233.456428
Age_Marital Status_Target                       138.773508
Age_Occupation_Relationship                     100.612767
Age_Occupation_Target                           101.319829
Age_Relationship_Race                           128.265933
Age_Relationship_Sex                            217.802323
Age_Relationship_Capital Gain                   110.820129
Age_Relationship_Capital Loss                   124.298635
Age_Relationship_Hours per week                 102.695111
Age_Relationship_Country                        103.880784
Age_Relationship_Target                         239.147452
Age_Race_Target                                 122.462705
Age_Sex_Target                                  214.120647
Workclass_Marital Status_Relationship           644.158365
Workclass_Marital Status_Target                 131.3635